# Speedtest data exploration

### How data is  collected

Running speedtest application (?) similar to speedtest.net Calculating upload, download speed + png latency

### How data looks like
Load libraries:

In [1]:
from data_exploration import *

Set up test time interval:

In [2]:
time_interval='4w' #5d

Set up influxdb connection:

In [3]:
client, client_df = connect_to_influxdb()

Checking last 10 records for the SPEEDTEST_UPLOAD  measurment and see how data looks like:

In [4]:
query_upload = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' ORDER BY time DESC LIMIT 10;"
upload_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload,table_name='SPEEDTEST_UPLOAD')
upload_df

time               IP             PI_MAC  \
9 2019-01-18 22:30:18  199.116.232.249  02-99-03-82-52-06   
4 2019-01-18 22:30:13   206.45.204.107  02-0e-04-02-e2-e0   
2 2019-01-18 22:30:10     204.9.14.110  02-0e-04-82-02-40   
3 2019-01-18 22:30:12   66.225.139.217  02-8d-07-82-34-1a   
0 2019-01-18 22:30:09      38.64.198.5  02-d8-06-c2-53-e7   
8 2019-01-18 22:30:14    199.19.63.140  02-84-05-80-b0-f2   
6 2019-01-18 22:30:13   162.249.42.115  02-08-02-c1-c1-34   
5 2019-01-18 22:30:13  204.112.104.114  02-cd-07-40-b1-3f   
1 2019-01-18 22:30:09   216.130.92.150  02-43-08-c0-b1-81   
7 2019-01-18 22:30:14   206.188.114.98  02-06-09-01-32-ac   

                       PROVIDER PROVINCE  SK_PI  \
9                        Cybera      AB)      3   
4                      Bell MTS      MB)      5   
2       Xplornet Communications      MB)      9   
3               TeraGo Networks      MB)     10   
0         Cogent Communications      MB)     11   
8               High Speed Crow      MB)     12   
6  Manitoba Hydro International      MB)     14   
5                      Bell MTS      MB)     15   
1                      Bell MTS      MB)     16   
7       Keewaytinook Okimakanak      MB)     17   

                        TEST_SERVER  UPLOAD  
9         Axia Connect Ltd (Calgary  127.02  
4                 LES.NET (Winnipeg    0.40  
2                RFNow Inc. (Virden   56.32  
3       Voyageur Internet (Winnipeg    3.41  
0       Voyageur Internet (Winnipeg   58.36  
8     Shaw Communications (Winnipeg    7.51  
6   Westman Communications (Brandon  132.80  
5  Fastnet Communications (Winnipeg   41.17  
1   Westman Communications (Brandon    3.20  
7   Westman Communications (Brandon   49.62

Let's take just one device, for example 3:

In [5]:
query_upload = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND SK_PI='3' ORDER BY time DESC LIMIT 10;"
upload_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload,table_name='SPEEDTEST_UPLOAD')
upload_df

time               IP             PI_MAC PROVIDER PROVINCE  \
0 2019-01-17 13:12:16  199.116.232.249  02-99-03-82-52-06   Cybera      AB)   
1 2019-01-17 16:54:10  199.116.232.249  02-99-03-82-52-06   Cybera      AB)   
2 2019-01-17 20:36:11  199.116.232.249  02-99-03-82-52-06   Cybera      AB)   
3 2019-01-18 00:18:08  199.116.232.249  02-99-03-82-52-06   Cybera      AB)   
4 2019-01-18 04:00:13  199.116.232.249  02-99-03-82-52-06   Cybera      AB)   
5 2019-01-18 07:42:09  199.116.232.249  02-99-03-82-52-06   Cybera      AB)   
6 2019-01-18 11:24:10  199.116.232.249  02-99-03-82-52-06   Cybera      AB)   
7 2019-01-18 15:06:14  199.116.232.249  02-99-03-82-52-06   Cybera      AB)   
8 2019-01-18 18:48:07  199.116.232.249  02-99-03-82-52-06   Cybera      AB)   
9 2019-01-18 22:30:18  199.116.232.249  02-99-03-82-52-06   Cybera      AB)   

   SK_PI                TEST_SERVER  UPLOAD  
0      3            Cybera (Calgary  128.08  
1      3            Cybera (Calgary  126.56  
2      3            Cybera (Calgary  127.86  
3      3            Cybera (Calgary  128.09  
4      3  Axia Connect Ltd (Calgary  126.23  
5      3  Axia Connect Ltd (Calgary  123.76  
6      3  Axia Connect Ltd (Calgary  123.85  
7      3  Axia Connect Ltd (Calgary  126.20  
8      3  Axia Connect Ltd (Calgary  122.73  
9      3  Axia Connect Ltd (Calgary  127.02

Let's compare with what we have in MS SQL database:

In [6]:
cnxn = connect_to_mssql()
sql = "SELECT TOP 10 * FROM FCT_SPEEDTEST WHERE PROVIDER!='iperf' AND SK_PI='3' ORDER BY DATA_DATE DESC;"
pd.read_sql(sql,cnxn)

SK_PI  SK_FILE_PATTERN               DATA_DATE PROVIDER               IP  \
0      3               19 2019-01-19 20:42:05.527   Cybera  199.116.232.249   
1      3               19 2019-01-19 17:00:06.207   Cybera  199.116.232.249   
2      3               19 2019-01-19 13:18:05.887   Cybera  199.116.232.249   
3      3               19 2019-01-19 09:36:06.567   Cybera  199.116.232.249   
4      3               19 2019-01-19 05:54:14.253   Cybera  199.116.232.249   
5      3               19 2019-01-19 02:12:16.937   Cybera  199.116.232.249   
6      3               19 2019-01-18 22:30:18.617   Cybera  199.116.232.249   
7      3               19 2019-01-18 18:48:07.287   Cybera  199.116.232.249   
8      3               19 2019-01-18 15:06:14.973   Cybera  199.116.232.249   
9      3               19 2019-01-18 11:24:10.650   Cybera  199.116.232.249   

                 TEST_SERVER PROVINCE    PING  DOWNLOAD  UPLOAD  
0  Axia Connect Ltd (Calgary      AB)  29.813    180.60  122.55  
1  Axia Connect Ltd (Calgary      AB)  13.500    181.75  123.92  
2  Axia Connect Ltd (Calgary      AB)  32.186    180.32  124.90  
3  Axia Connect Ltd (Calgary      AB)  29.731    179.62  122.64  
4  Axia Connect Ltd (Calgary      AB)  13.443    180.78  125.50  
5  Axia Connect Ltd (Calgary      AB)  29.293    178.86  125.76  
6  Axia Connect Ltd (Calgary      AB)  10.062    178.48  127.02  
7  Axia Connect Ltd (Calgary      AB)  13.301    184.52  122.73  
8  Axia Connect Ltd (Calgary      AB)  13.481    181.79  126.20  
9  Axia Connect Ltd (Calgary      AB)  13.409    179.98  123.85

Are there any zeros in mssql database?

In [7]:
sql = "SELECT * FROM FCT_SPEEDTEST WHERE PROVIDER!='iperf' AND (UPLOAD=0 OR DOWNLOAD=0 OR PING=0);"
pd.read_sql(sql,cnxn)

SK_PI  SK_FILE_PATTERN               DATA_DATE  PROVIDER             IP  \
0      7               19 2018-12-13 20:42:03.723  Bell MTS  204.112.96.66   
1      4               19 2018-11-24 21:36:08.367  Bell MTS  204.112.96.94   
2      4               19 2018-11-27 05:06:15.177  Bell MTS  204.112.96.94   
3      4               19 2018-11-30 14:30:05.207  Bell MTS  204.112.96.94   

                           TEST_SERVER PROVINCE         PING  DOWNLOAD  UPLOAD  
0  Commstream Communications (Winnipeg      MB)  1800000.000      0.00    0.00  
1          Voyageur Internet (Winnipeg      MB)       67.381      1.19    0.00  
2        Shaw Communications (Winnipeg      MB)       68.232      0.00    0.65  
3        Shaw Communications (Winnipeg      MB)       78.877      1.01    0.00

Something is possibly wrong with these measurments, we wille exclude zeros for now.

### How may data points per device?

Getting device numbers(tags SK_PI):

In [8]:
device_numbers=get_tag_values_influxdb(client_influx=client,table_name='SPEEDTEST_UPLOAD', tag_name='SK_PI')
device_numbers=list(map(int, device_numbers))
device_numbers= sorted(device_numbers)
print(device_numbers)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18]


Getting number of data points per device for the entire period of time.

In [9]:
query_upload_counts = "SELECT COUNT(UPLOAD) FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND UPLOAD>0 GROUP BY SK_PI;"
upload_counts=get_stats_influxdb(client_influx=client,
                               query_influx=query_upload_counts,
                               stat_name='count',
                               device_numbers=device_numbers)

Plotting device numbers and number of data points. 

In [10]:
simple_bar_plot(xvalues=device_numbers,
                yvalues=upload_counts,
                name="upload datapoints",
                title="Number of data points per device",
                ytitle="Number of datapoints")

Some of the devices have small number of datapoints, may be they are just installed? Lets check how many dataponts came in last 4 weeks.

Getting number of datapoints per device in last 4 weeks.

In [11]:
query_upload_counts_time = "SELECT COUNT(UPLOAD) FROM SPEEDTEST_UPLOAD WHERE time >= now()-"+time_interval+" AND PROVIDER!='iperf' AND UPLOAD>0 GROUP BY SK_PI ;"
upload_counts_time = get_stats_influxdb(client_influx=client,
                                      query_influx=query_upload_counts_time,
                                      stat_name='count',
                                      device_numbers=device_numbers)

Plotting combined barchart - entire number of datapoints vs number of datapoints in last 4 weeks.

In [12]:
combined_bar_plot_2traces(xvalues=device_numbers,
                          yvalues1=upload_counts_time,
                          yvalues2=[a - b for a, b in zip(upload_counts, upload_counts_time)],
                          name1='Last '+time_interval,
                          name2='The rest of the time',
                          title="Comparing number of datapoints in last "+time_interval+" vs entire time",
                          ytitle="Number of datapoints")

Devices 1,2,4,6 and 8 have data but not in the last 4 weeks.Let's check last reporting time for every device.

In [13]:
query_upload_last = "SELECT LAST(UPLOAD), time FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND UPLOAD>0 GROUP BY SK_PI;"
result_upload_last=get_stats_influxdb(client_influx=client,
                               query_influx=query_upload_last,
                               stat_name='time',
                               device_numbers=device_numbers)

In [14]:
query_upload_first = "SELECT FIRST(UPLOAD), time FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND UPLOAD>0 GROUP BY SK_PI;"
result_upload_first=get_stats_influxdb(client_influx=client,
                               query_influx=query_upload_first,
                               stat_name='time',
                               device_numbers=device_numbers)

In [15]:
print("Speedtest reporting times:")
data=[]
for i in range(len(device_numbers)):
    try:
        result_upload_first[i] = dateutil.parser.parse(result_upload_first[i]).strftime('%Y-%m-%d %H:%M:%S')
    except:
        result_upload_first[i]=None
    try:    
        result_upload_last[i] = dateutil.parser.parse(result_upload_last[i]).strftime('%Y-%m-%d %H:%M:%S')
    except:
        result_upload_last[i]=None
    print("Device: ", device_numbers[i],"  was reporting from ", result_upload_first[i], " to ",result_upload_last[i])
    trace = go.Scatter(x=[result_upload_first[i],result_upload_last[i]],y=[device_numbers[i],device_numbers[i]], 
                       name = device_numbers[i],marker=dict(color=colors[i]))
    data.append(trace)
layout = dict(title = "Device reporting times(speedtest)",xaxis=dict(title="Time"),
        yaxis=dict(title="Device Number"))
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Speedtest reporting times:
Device:  1   was reporting from  2018-10-10 15:45:43  to  2018-10-11 12:00:09
Device:  2   was reporting from  2018-10-11 15:55:58  to  2018-11-03 15:00:12
Device:  3   was reporting from  2018-10-15 19:36:18  to  2019-01-18 22:30:18
Device:  4   was reporting from  2018-10-27 00:03:48  to  2018-12-05 16:36:12
Device:  5   was reporting from  2018-10-26 18:18:08  to  2019-01-18 22:30:13
Device:  6   was reporting from  2018-10-27 20:12:15  to  2018-10-31 13:00:10
Device:  7   was reporting from  2018-11-11 00:36:13  to  2019-01-18 22:30:05
Device:  8   was reporting from  2018-11-07 18:22:07  to  2019-01-14 11:12:13
Device:  9   was reporting from  2018-11-30 18:12:12  to  2019-01-18 22:30:10
Device:  10   was reporting from  2018-12-12 22:29:40  to  2019-01-18 22:30:12
Device:  11   was reporting from  2018-11-30 18:12:09  to  2019-01-18 22:30:09
Device:  12   was reporting from  2018-11-30 18:12:06  to  2019-01-18 22:30:14
Device:  14   was reporting from  

### Statistics by device

In [16]:
query_download = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER!='iperf'AND DOWNLOAD>0 AND time >= now()-"+\
                         time_interval;
download_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_download,
                                          table_name='SPEEDTEST_DOWNLOAD')

In [17]:
query_upload = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf'AND UPLOAD>0 AND time >= now()-"+\
                         time_interval;
upload_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload,
                                          table_name='SPEEDTEST_UPLOAD')

In [18]:
query_ping = "SELECT * FROM SPEEDTEST_PING WHERE PROVIDER!='iperf'AND PING>0 AND time >= now()-"+\
                         time_interval;
ping_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_ping,
                                          table_name='SPEEDTEST_PING')
ping_df = ping_df[ping_df.PING != 1800000.000] ## Removed outier that was found in the ms sql table

In [19]:
download_summary=mean_max_median_by1(download_df,'DOWNLOAD')
download_line=go.Scatter(x=device_numbers,y=[50] * len(device_numbers), mode='markers',marker=dict(color='red'), name='50Mps')
combined_bar_plot_3traces(xvalues=download_summary["SK_PI"],
                         yvalues1=download_summary["max"],
                         yvalues2=download_summary["mean"],
                         yvalues3=download_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Download speed over the last "+time_interval,
                         ytitle="Mbps",
                         line=download_line,
                         stack=False)

In [20]:
upload_line=go.Scatter(x=device_numbers,y=[10] * len(device_numbers), mode='markers',marker=dict(color='red'), name='10Mps')
upload_summary=mean_max_median_by1(upload_df,'UPLOAD')
combined_bar_plot_3traces(xvalues=upload_summary["SK_PI"],
                         yvalues1=upload_summary["max"],
                         yvalues2=upload_summary["mean"],
                         yvalues3=upload_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Upload speed over the last "+time_interval,
                         ytitle="Mbps",
                         line=upload_line,
                         stack=False)

In [21]:
ping_summary=mean_max_median_by1(ping_df,'PING')
combined_bar_plot_3traces(xvalues=ping_summary["SK_PI"],
                         yvalues1=ping_summary["max"],
                         yvalues2=ping_summary["mean"],
                         yvalues3=ping_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Ping latency over the last "+time_interval,
                         ytitle="Miliseconds",
                         stack=False)

In [22]:
simple_boxplot(dataframe=download_df,plot_value='DOWNLOAD',sort_value='SK_PI',
               title="Download speed over the last "+time_interval, 
               ytitle="Mbps")

In [23]:
simple_boxplot(dataframe=upload_df,plot_value='UPLOAD',sort_value='SK_PI',
               title="Upload speed over the last "+time_interval, 
               ytitle="Mbps")

In [24]:
simple_boxplot(dataframe=ping_df,plot_value='PING',sort_value='SK_PI',
               title="Ping latency over the last "+time_interval, 
               ytitle="Miliseconds")

### Test servers
Are there any NaNs or empty strings?

In [25]:
print(sum(pd.isnull(upload_df["TEST_SERVER"])))
print(sum(pd.isnull(upload_df["PROVINCE"])))
print(sum(upload_df["TEST_SERVER"] == ''))
print(sum(upload_df["PROVINCE"] == ''))

0
0
0
0


In [26]:
print("There are ",len(upload_df["PROVINCE"].unique()), " provinces: ", upload_df["PROVINCE"].unique(), "\n")
print("There are ",len(upload_df["TEST_SERVER"].unique()), " test servers", upload_df["TEST_SERVER"].unique(), "\n")

pd.options.mode.chained_assignment = None
upload_df["server"] = upload_df["TEST_SERVER"] + " " +upload_df["PROVINCE"]
print("There are ",len(upload_df["server"].unique()), " test servers+provinces", upload_df["server"].unique(), "\n")

There are  4  provinces:  [' AB)' ' MB)' ' SK)' ' ND)'] 

There are  21  test servers ['Axia Connect Ltd (Calgary' 'DataHive (Calgary' 'Cybera (Calgary'
 'Rogers (Winnipeg' 'Voyageur Internet (Winnipeg'
 'Shaw Communications (Winnipeg' 'LES.NET (Winnipeg'
 'Commstream Communications (Winnipeg' 'Fastnet Communications (Winnipeg'
 'Valley Fiber (Winkler' 'VOI Network Solutions (Winnipeg'
 'NetSet Communications (Brandon' 'Westman Communications (Brandon'
 'Morenet (Morden' 'BlackSun Inc. (Saskatoon' 'SaskTel (Regina'
 'RFNow Inc. (Virden' 'Access Communications Co-operative Limited (Regina'
 'Bell Mobility (Winnipeg' 'Midco (Minot' 'RFNow Inc (Brandon'] 

There are  21  test servers+provinces ['Axia Connect Ltd (Calgary  AB)' 'DataHive (Calgary  AB)'
 'Cybera (Calgary  AB)' 'Rogers (Winnipeg  MB)'
 'Voyageur Internet (Winnipeg  MB)' 'Shaw Communications (Winnipeg  MB)'
 'LES.NET (Winnipeg  MB)' 'Commstream Communications (Winnipeg  MB)'
 'Fastnet Communications (Winnipeg  MB)' 'Valley Fi

#### How many test servers per device?

In [27]:
pd.options.mode.chained_assignment = None
upload_df["server"] = upload_df["TEST_SERVER"] + " " +upload_df["PROVINCE"]
download_df["server"] = download_df["TEST_SERVER"] + " " +download_df["PROVINCE"]
ping_df["server"] = ping_df["TEST_SERVER"] + " " +ping_df["PROVINCE"]

points_by_device=upload_df.groupby(['SK_PI']).size().reset_index(name='counts').sort_values('SK_PI')

test_servers_per_device = upload_df.groupby('SK_PI').apply(lambda x: x['server'].unique()).apply(pd.Series)
test_servers_per_device['server_count'] = test_servers_per_device.apply(lambda x: x.count(), axis=1)
test_server_counts_per_device = pd.Series(test_servers_per_device['server_count']).value_counts().reset_index()
points_by_device_by_server=upload_df.groupby(['SK_PI', 'server']).size().reset_index()

In [28]:
simple_bar_plot(xvalues=test_servers_per_device.index,
                yvalues=test_servers_per_device["server_count"],
                name = "Number of providers",title="Number of test servers per device over the last "+time_interval,
                ytitle="Number of providers")

In [29]:
simple_pie_chart(labels=test_server_counts_per_device['index'],
            values=test_server_counts_per_device['server_count'],
                 title="Number of test servers per device over the last "+time_interval)

In [30]:
combined_bar_plot_multitraces(dataframe=points_by_device_by_server,
                              device_numbers=device_numbers,
                              points_by_device=points_by_device,
                              sort_value='server',
                              ytitle="Percentage of data points",
                              title="Percentage of data points per device per test server over the last "+time_interval)

In [31]:
upload_summary=mean_max_median_by1(upload_df,'UPLOAD',index_col='server')
upload_line=go.Scatter(x=upload_summary["server"].unique(),y=[10] * len(upload_summary["server"].unique()), mode='markers',marker=dict(color='red'), name='10Mps')
combined_bar_plot_3traces(xvalues=upload_summary["server"],
                         yvalues1=upload_summary["max"],
                         yvalues2=upload_summary["mean"],
                         yvalues3=upload_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Upload speed by test server over the last "+time_interval,
                         ytitle="Mbps",
                         xtitle="Provider",
                         line=upload_line,
                         stack=False,
                         margin=True)

In [32]:
download_summary=mean_max_median_by1(download_df,'DOWNLOAD',index_col='server')
download_line=go.Scatter(x=upload_summary['server'].unique(),y=[50] * len(upload_summary['server'].unique()), mode='markers',marker=dict(color='red'), name='50Mps')
combined_bar_plot_3traces(xvalues=download_summary['server'],
                         yvalues1=download_summary["max"],
                         yvalues2=download_summary["mean"],
                         yvalues3=download_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Download speed by test server over the last "+time_interval,
                         ytitle="Mbps",
                         xtitle="Test server",
                         line=download_line,
                         stack=False,
                         margin=True)

In [33]:
ping_summary=mean_max_median_by1(ping_df,'PING',index_col='server')
combined_bar_plot_3traces(xvalues=ping_summary['server'],
                         yvalues1=ping_summary["max"],
                         yvalues2=ping_summary["mean"],
                         yvalues3=ping_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Ping latency by  test server over the last "+time_interval,
                         ytitle="Miliseconds",
                         xtitle="Test server",
                         stack=False,
                         margin=True)

### Providers
Are there any NaNs or empty strings?

In [34]:
print(sum(pd.isnull(upload_df["PROVIDER"])))
print(sum(upload_df["PROVIDER"] == ''))

0
0


In [35]:
print("There are ",len(upload_df["PROVIDER"].unique()), " providers", upload_df["PROVIDER"].unique(), "\n")

There are  10  providers ['Cybera' 'Bell MTS' 'Commstream Communications' 'MERLIN'
 'Xplornet Communications' 'TeraGo Networks' 'Cogent Communications'
 'High Speed Crow' 'Manitoba Hydro International'
 'Keewaytinook Okimakanak'] 



#### How many providers per device?

In [36]:
points_by_device=upload_df.groupby(['SK_PI']).size().reset_index(name='counts').sort_values('SK_PI')
providers_per_device = upload_df.groupby('SK_PI').apply(lambda x: x["PROVIDER"].unique()).apply(pd.Series)
providers_per_device['provider_count'] = providers_per_device.apply(lambda x: x.count(), axis=1)
provider_counts_per_device = pd.Series(providers_per_device['provider_count']).value_counts().reset_index()
points_by_device_by_provider=upload_df.groupby(['SK_PI', 'PROVIDER']).size().reset_index()

In [37]:
simple_bar_plot(xvalues=providers_per_device.index,
                yvalues=providers_per_device["provider_count"],
                name = "Number of providers",title="Number of providers per device over the last "+time_interval,
                ytitle="Number of providers")

In [38]:
simple_pie_chart(labels=provider_counts_per_device['index'],
            values=provider_counts_per_device["provider_count"],title="Number of providers per device over the last "+time_interval)

In [39]:
combined_bar_plot_multitraces(dataframe=points_by_device_by_provider,
                              device_numbers=device_numbers,
                              points_by_device=points_by_device,
                              sort_value='PROVIDER',
                              ytitle="Percentage of data",
                              title="Percentage of data points per device per service provider over the last "+time_interval)

In [40]:
upload_summary=mean_max_median_by1(upload_df,'UPLOAD',index_col='PROVIDER')
upload_line=go.Scatter(x=upload_summary["PROVIDER"].unique(),y=[10] * len(upload_summary["PROVIDER"].unique()), mode='markers',marker=dict(color='red'), name='10Mps')
combined_bar_plot_3traces(xvalues=upload_summary["PROVIDER"],
                         yvalues1=upload_summary["max"],
                         yvalues2=upload_summary["mean"],
                         yvalues3=upload_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Upload speed by provider over the last "+time_interval,
                         ytitle="Mbps",
                         xtitle="Provider",
                         line=upload_line,
                         stack=False,
                         margin=True)

In [41]:
download_summary=mean_max_median_by1(download_df,'DOWNLOAD',index_col='PROVIDER')
download_line=go.Scatter(x=upload_summary["PROVIDER"].unique(),y=[50] * len(upload_summary["PROVIDER"].unique()), mode='markers',marker=dict(color='red'), name='50Mps')
combined_bar_plot_3traces(xvalues=download_summary["PROVIDER"],
                         yvalues1=download_summary["max"],
                         yvalues2=download_summary["mean"],
                         yvalues3=download_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Download speed by provider over the last "+time_interval,
                         ytitle="Mbps",
                         xtitle="Provider",
                         line=download_line,
                         stack=False,
                         margin=True)

In [42]:
ping_summary=mean_max_median_by1(ping_df,'PING',index_col='PROVIDER')
combined_bar_plot_3traces(xvalues=ping_summary["PROVIDER"],
                         yvalues1=ping_summary["max"],
                         yvalues2=ping_summary["mean"],
                         yvalues3=ping_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Ping latency by provider over the last "+time_interval,
                         ytitle="Miliseconds",
                         xtitle="Provider",
                         stack=False,
                         margin=True)

### How often data is collected?

Let's ceck device number 5 with the larges number of datapoints and (8 test servers, 2 providers)  and device number 8 with largest number of providers (3) and 4 test servers?

In [43]:
upload_df.loc[upload_df['SK_PI']==5][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].head(5)

time  SK_PI  PROVIDER                            server  \
8  2018-12-24 04:36:14      5  Bell MTS             Rogers (Winnipeg  MB)   
20 2018-12-24 08:18:14      5  Bell MTS             Rogers (Winnipeg  MB)   
30 2018-12-24 12:00:10      5  Bell MTS             Rogers (Winnipeg  MB)   
37 2018-12-24 15:42:06      5  Bell MTS  Voyageur Internet (Winnipeg  MB)   
58 2018-12-24 19:24:13      5  Bell MTS             Rogers (Winnipeg  MB)   

    UPLOAD  
8     0.47  
20    0.53  
30    0.49  
37    0.49  
58    0.40

In [44]:
upload_df.loc[upload_df['SK_PI']==5][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].tail(5)

time  SK_PI  PROVIDER  \
1956 2019-01-18 07:42:19      5  Bell MTS   
1967 2019-01-18 11:24:14      5  Bell MTS   
1978 2019-01-18 15:06:13      5  Bell MTS   
1990 2019-01-18 18:48:14      5  Bell MTS   
2000 2019-01-18 22:30:13      5  Bell MTS   

                                     server  UPLOAD  
1956                 LES.NET (Winnipeg  MB)    0.49  
1967                  Rogers (Winnipeg  MB)    0.53  
1978                  Rogers (Winnipeg  MB)    0.50  
1990  Fastnet Communications (Winnipeg  MB)    0.48  
2000                 LES.NET (Winnipeg  MB)    0.40

In [45]:
upload_df.loc[upload_df['SK_PI']==8][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].head(5)

time  SK_PI                   PROVIDER  \
1361 2019-01-10 20:27:41      8  Commstream Communications   
1378 2019-01-11 01:48:11      8                     MERLIN   
1386 2019-01-11 05:30:05      8  Commstream Communications   
1403 2019-01-11 09:12:08      8  Commstream Communications   
1420 2019-01-11 12:54:12      8                     MERLIN   

                                    server  UPLOAD  
1361      Voyageur Internet (Winnipeg  MB)   84.39  
1378  VOI Network Solutions (Winnipeg  MB)   89.90  
1386      Voyageur Internet (Winnipeg  MB)  120.97  
1403      Voyageur Internet (Winnipeg  MB)   89.75  
1420  VOI Network Solutions (Winnipeg  MB)   90.05

In [46]:
upload_df.loc[upload_df['SK_PI']==5][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].tail(5)

time  SK_PI  PROVIDER  \
1956 2019-01-18 07:42:19      5  Bell MTS   
1967 2019-01-18 11:24:14      5  Bell MTS   
1978 2019-01-18 15:06:13      5  Bell MTS   
1990 2019-01-18 18:48:14      5  Bell MTS   
2000 2019-01-18 22:30:13      5  Bell MTS   

                                     server  UPLOAD  
1956                 LES.NET (Winnipeg  MB)    0.49  
1967                  Rogers (Winnipeg  MB)    0.53  
1978                  Rogers (Winnipeg  MB)    0.50  
1990  Fastnet Communications (Winnipeg  MB)    0.48  
2000                 LES.NET (Winnipeg  MB)    0.40

 Looks like data is collected with 3 hours 42 mins intervals using different test servers. 
 How is it determined which test server to use?

Bell MTS has largest maximum spikes and TerraGo has largest median/average. 